# 1) Preparación previa

### Carga de librerías

In [75]:
# Se debe instalar ipyleaflet, mpu, openpyxl y requests

import requests
from pandas import json_normalize
import ipyleaflet
import mpu
import pandas as pd
import numpy as np
from ipyleaflet import Map, basemaps, Marker, AwesomeIcon, Icon, FullScreenControl

### Íconos a usar luego

In [76]:
# Los siguientes íconos serán utilizados para visualizar en el mapa final 

icono_subte = Icon(icon_url='https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Subte-logo.svg/1200px-Subte-logo.svg.png', icon_size=[28, 28])

icon0 = AwesomeIcon(
    name='home',
    marker_color='green',
    icon_color='black',
    spin=False
)

icon1 = AwesomeIcon(
    name='home',
    marker_color='beige',
    icon_color='black',
    spin=False
)

icon2 = AwesomeIcon(
    name='home',
    marker_color='orange',
    icon_color='black',
    spin=False
)

icon3 = AwesomeIcon(
    name='home',
    marker_color='red',
    icon_color='black',
    spin=False
)

### Estaciones a usar luego

In [77]:
# Dirección web para acceder a los datos del gobierno de la ciudad:
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/subte-estaciones/subte_estaciones.geojson'
response = requests.get(url)
dictr = response.json()

# Hacemos referencia a las estaciones dentro del conjunto de datos:
estaciones = dictr['features']
df = json_normalize(estaciones)

# Creamos la columna Longitud:
df['Longitud'] = df['geometry.coordinates'].apply(lambda x:x[0])

# Creamos la columna Latitud:
df['Latitud'] = df['geometry.coordinates'].apply(lambda x:x[1])

# Creamos la máscara para filtrar los de Linea A:
mascara_lineas_elegidas = df['properties.LINEA'].isin(['A', 'B', 'C', 'E', 'H']) 
lineas_elegidas = df.loc[mascara_lineas_elegidas, :]
lineas_elegidas = lineas_elegidas.drop(['type', 'properties.ID', 'properties.LINEA', 'geometry.type', 'geometry.coordinates'], axis = 1)
lineas_elegidas

properties.ESTACION   Longitud    Latitud
0                                 CASEROS -58.398928 -34.635750
1              INCLAN - MEZQUITA AL AHMAD -58.400970 -34.629376
2                             HUMBERTO 1° -58.402323 -34.623092
3                               VENEZUELA -58.404732 -34.615242
4                  ONCE - 30 DE DICIEMBRE -58.406036 -34.608935
..                                    ...        ...        ...
85             SANTA FE - CARLOS JAUREGUI -58.402376 -34.594525
86  FACULTAD DE DERECHO - JULIETA LANTERI -58.391019 -34.583036
87                                 RETIRO -58.375850 -34.592114
88                              CATALINAS -58.371700 -34.596597
89                         CORREO CENTRAL -58.370413 -34.603014

[74 rows x 3 columns]

### Funciones a usar luego

In [78]:
# La siguiente función nos permite definir la distancia en latitud y longitud respecto de las líneas de subte:
def distancia(lat, long):
    aux = []
    for index, row in lineas_elegidas.iterrows():
        aux.append(
            mpu.haversine_distance(
                (row['Latitud'], row['Longitud']), 
                (lat, long)
            )
        )
    return min(aux)

# La siguiente función nos permite aplicar íconos a los marcadores según su quantil:
def marca_propiedad(lat, long, quantile):
    if quantile == 0:
        icono_prop = icon0
    elif quantile == 1:
        icono_prop = icon1
    elif quantile == 2:
        icono_prop = icon2  
    else: 
        icono_prop = icon3
    marker = Marker(location=(lat,long), draggable=False, icon=icono_prop)
    basic_map.add_layer(marker)

# La siguiente función nos permite limpiar la superficie según las inconsistencias entre la total y la cubierta.
# Se eligií dicha columna con respecto a 'surface_total_in_m2' ya que tenía menor cantidad de registros nulos
def limpieza_superficie(sup_total, sup_cubierta):
    if sup_total is not None and sup_cubierta is not None:
            if sup_total >= sup_cubierta:
                superficie = sup_total
            else: 
                superficie = np.NaN
    elif sup_total is not None:
            superficie = sup_total
    elif sup_cubierta is not None:
            superficie = sup_cubierta
    else: superficie = np.NaN             
                    
    return superficie

### Lectura del dataset original de Properati

In [79]:
df_properati = pd.read_csv('properati.csv')

# 2) Limpieza del dataset

### Generación de DF solo para Capital

In [83]:
df_capital = df_properati[df_properati["state_name"] == "Capital Federal"]
df_capital.shape

(32316, 26)

### Corrección de ortografía de barrios y mapeo de barrios inexistentes

In [84]:
%%capture output
df_capital["place_name"].replace(['San Cristobal','Barrio Norte', 'Centro / Microcentro'], ['San Cristóbal','Recoleta', 'Monserrat'], inplace=True)

### Lectuura el archivo de Mapping de Barrios por Comuna

In [69]:
df_barrios_comuna = pd.read_csv("https://raw.githubusercontent.com/Grupo8DH/ProyectoProperati/main/Barrios_x_Comuna_vsc_utf8.csv")

### Merge del DF de Capital con los barrios según comunas

In [70]:
df_capital_comuna = pd.merge(df_capital, df_barrios_comuna, how='left', left_on='place_name', right_on='Barrio')
del df_capital_comuna["description"]
del df_capital_comuna["title"]
del df_capital_comuna["properati_url"]
del df_capital_comuna["image_thumbnail"]


In [71]:
df_capital_comuna["Comuna"].isna().sum()

3612

### Enumeración de los barrios por los que atraviesa la Linea A, elegida como la principal para analizar.

In [51]:
barrios_linea_a = pd.Series(['Monserrat', 'Balvanera', 'Almagro', 'Caballito'])
barrios_linea_a

0    Monserrat
1    Balvanera
2      Almagro
3    Caballito
dtype: object

### Filtro del DF de Capital según los barrios definidos

In [52]:
df_capital_barrios_subte = df_capital_comuna[df_capital_comuna['Barrio'].isin(barrios_linea_a)]
df_capital_barrios_subte.shape

(4866, 24)

### Eliminación de los registros sin coordenadas

In [53]:
df_latlon_notna = df_capital_barrios_subte[df_capital_barrios_subte['lat-lon'].notna()]

In [54]:
df_latlon_notna.shape

(4031, 24)

### Eliminación de los registros sin precios

In [85]:
df_price_notna = df_latlon_notna[df_latlon_notna['price_usd_per_m2'].notna()]
df_price_notna

Unnamed: 0 operation property_type place_name  \
14             48      sell     apartment  Balvanera   
16             52      sell     apartment  Caballito   
17             53      sell     apartment  Caballito   
18             54      sell     apartment  Caballito   
19             55      sell     apartment  Caballito   
...           ...       ...           ...        ...   
31793      119970      sell     apartment  Balvanera   
31817      120075      sell     apartment  Caballito   
32198      120984      sell     apartment  Caballito   
32267      121059      sell     apartment  Monserrat   
32273      121075      sell     apartment  Caballito   

                     place_with_parent_names country_name       state_name  \
14     |Argentina|Capital Federal|Balvanera|    Argentina  Capital Federal   
16     |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
17     |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
18     |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
19     |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
...                                      ...          ...              ...   
31793  |Argentina|Capital Federal|Balvanera|    Argentina  Capital Federal   
31817  |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
32198  |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   
32267  |Argentina|Capital Federal|Monserrat|    Argentina  Capital Federal   
32273  |Argentina|Capital Federal|Caballito|    Argentina  Capital Federal   

       geonames_id                        lat-lon        lat  ...  \
14       6693228.0        -34.6096748,-58.3986588 -34.609675  ...   
16       3435874.0        -34.6222111,-58.4391283 -34.622211  ...   
17       3435874.0  -34.6238943143,-58.4468606529 -34.623894  ...   
18       3435874.0        -34.6209278,-58.4458736 -34.620928  ...   
19       3435874.0        -34.6209278,-58.4458736 -34.620928  ...   
...            ...                            ...        ...  ...   
31793    6693228.0        -34.6096441,-58.3964932 -34.609644  ...   
31817    3435874.0  -34.6221978289,-58.4291563359 -34.622198  ...   
32198    3435874.0  -34.6204033093,-58.4410224908 -34.620403  ...   
32267    3430570.0   -34.613408336,-58.3845426597 -34.613408  ...   
32273    3435874.0   -34.627466548,-58.4386614795 -34.627467  ...   

       surface_covered_in_m2  price_usd_per_m2 price_per_m2  floor  rooms  \
14                     124.0       1702.127660  1935.483871    NaN    5.0   
16                      50.0       2200.000000  2200.000000    NaN    3.0   
17                      46.0       1705.000000  3706.521739    NaN    2.0   
18                      75.0       2721.111111  2938.800000    NaN    3.0   
19                      75.0       2750.370370  2970.400000    NaN    3.0   
...                      ...               ...          ...    ...    ...   
31793                   38.0       2631.111111  3115.789474    NaN    1.0   
31817                   70.0       2051.282051  2285.714286    NaN    NaN   
32198                   77.0       1909.090909  1909.090909    NaN    NaN   
32267                   34.0       2310.810811  2514.705882    NaN    NaN   
32273                   83.0       3461.538462  3795.180723    NaN    NaN   

       expenses     Barrio    Comuna  distancia  superficie  
14          NaN  Balvanera  Comuna-3   0.021456       141.0  
16          NaN  Caballito  Comuna-6   0.274751        50.0  
17          NaN  Caballito  Comuna-6   0.168486       100.0  
18          NaN  Caballito  Comuna-6   0.384825        81.0  
19          NaN  Caballito  Comuna-6   0.384825        81.0  
...         ...        ...       ...        ...         ...  
31793       NaN  Balvanera  Comuna-3   0.176977        45.0  
31817    2300.0  Caballito  Comuna-6   0.577834        78.0  
32198    3800.0  Caballito  Comuna-6   0.014197        77.0  
32267    1200.0  Monser

### Limpieza de la superficie

In [ ]:
# Se debe corregir inconsistencias entre la superficie total y la cubierta

In [87]:
%%capture output
df_price_notna["superficie"] = df_price_notna[['surface_total_in_m2','surface_covered_in_m2']].apply(lambda df_price_notna: limpieza_superficie(df_price_notna['surface_total_in_m2'],df_price_notna['surface_covered_in_m2']),axis=1)
df_price_notna.head(4)

# 3) Análisis de hipótesis

### Cálculo de las distancias entre cada propiedad y las 17 estaciones

In [86]:
%%capture output
df_price_notna['distancia'] = df_price_notna.apply(lambda x: distancia(x['lat'], x['lon']), axis=1)
df_price_notna

### Sampleo aleatorio para obtener una visualización más prolija

In [59]:
df_randomizado = df_price_notna.sample(n=100, random_state = 1)

### Definición de quantiles

In [60]:
df_randomizado['quantile'] = pd.qcut(df_randomizado['price_usd_per_m2'], 4, labels=False)
df_randomizado

Unnamed: 0 operation property_type place_name  \
22423       81835      sell     apartment  Caballito   
17309       64627      sell     apartment  Caballito   
13749       52061      sell         store  Monserrat   
10967       42365      sell     apartment  Caballito   
1231         6751      sell         house    Almagro   
...           ...       ...           ...        ...   
20847       76673      sell     apartment  Caballito   
29288      111790      sell     apartment  Monserrat   
695          4643      sell     apartment  Caballito   
29947      113701      sell     apartment    Almagro   
18             54      sell     apartment  Caballito   

                                place_with_parent_names country_name  \
22423             |Argentina|Capital Federal|Caballito|    Argentina   
17309             |Argentina|Capital Federal|Caballito|    Argentina   
13749  |Argentina|Capital Federal|Centro / Microcentro|    Argentina   
10967             |Argentina|Capital Federal|Caballito|    Argentina   
1231                |Argentina|Capital Federal|Almagro|    Argentina   
...                                                 ...          ...   
20847             |Argentina|Capital Federal|Caballito|    Argentina   
29288             |Argentina|Capital Federal|Monserrat|    Argentina   
695               |Argentina|Capital Federal|Caballito|    Argentina   
29947               |Argentina|Capital Federal|Almagro|    Argentina   
18                |Argentina|Capital Federal|Caballito|    Argentina   

            state_name  geonames_id                        lat-lon        lat  \
22423  Capital Federal    3435874.0  -34.6316825892,-58.3574144339 -34.631683   
17309  Capital Federal    3435874.0  -34.6040302276,-58.4323198061 -34.604030   
13749  Capital Federal    3435548.0          -34.607924,-58.374815 -34.607924   
10967  Capital Federal    3435874.0         -34.6117495,-58.441553 -34.611750   
1231   Capital Federal    3436397.0          -34.603771,-58.381587 -34.603771   
...                ...          ...                            ...        ...   
20847  Capital Federal    3435874.0        -34.6277891,-58.4523659 -34.627789   
29288  Capital Federal    3430570.0  -34.6197919041,-58.3813379478 -34.619792   
695    Capital Federal    3435874.0        -34.6209278,-58.4458736 -34.620928   
29947  Capital Federal    3436397.0          -34.599744,-58.419144 -34.599744   
18     Capital Federal    3435874.0        -34.6209278,-58.4458736 -34.620928   

       ...  price_usd_per_m2  price_per_m2 floor  rooms  expenses     Barrio  \
22423  ...       3065.306452   3878.551020   NaN    NaN       NaN  Caballito   
17309  ...       2205.882353   2272.727273   NaN    NaN    2150.0  Caballito   
13749  ...       3033.707865   3033.707865   NaN    NaN       NaN  Monserrat   
10967  ...       3560.000000   3955.555556   NaN    2.0       NaN  Caballito   
1231   ...       1078.125000   1642.857143   NaN    NaN       NaN    Almagro   
...    ...               ...           ...   ...    ...       ...        ...   
20847  ...       3125.000000   3289.473684   6.0    3.0    1375.0  Caballito   
29288  ...       1955.555556   2155.102041   NaN    NaN    1000.0  Monserrat   
695    ...       1687.500000   1862.068966   NaN    5.0       NaN  Caballito   
29947  ...        978.260870   3214.285714   NaN    2.0       NaN    Almagro   
18     ...       2721.111111   2938.800000   NaN    3.0       NaN  Caballito   

         Comuna  distancia  superficie  quantile  
22423  Comuna-6   2.243691        62.0         3  
17309  Comuna-6   0.228711        34.0         1  
13749  Comuna-1   0.086576        89.0         3  
10967  Comuna-6   0.864416        50.0         3  
1231   Comuna-5   0.081200       640.0         0  
...         ...        ...         ...       ...  
20847  Comuna-6   0.416598        80.0         3  
29288  Comuna-1   0.206999        54.0         1  
695    Comuna-6   0.384825       160.0         0  
29947  Comuna-5   0.41520

### Visualización en un mapa

In [88]:
basic_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(-34.60823238941058, -58.398160858739075), zoom=14)

for index, row in lineas_elegidas.iterrows():
    marker = Marker(location=(row['Latitud'],row['Longitud']), draggable=False, icon=icono_subte)
    basic_map.add_layer(marker);

df_randomizado[['lat','lon','quantile']].apply(lambda x: marca_propiedad(x['lat'],x['lon'],x['quantile']),axis=1)
basic_map.add_control(FullScreenControl())

display(basic_map)

Map(center=[-34.60823238941058, -58.398160858739075], controls=(ZoomControl(options=['position', 'zoom_in_text…

### Exportación del .CSV

In [62]:
df_randomizado.to_csv("df_randomizado_CABA.csv", index=False)